# Task 2 - BioBERT based NER

## 1. Install necessary libraries

In [ ]:
!pip install -q contractions transformers sent2vec imbalanced-learn seqeval[gpu]
!pip install -q tf-estimator-nightly==2.8.0.dev2021122109
!python -m pip uninstall -q -y spacy
!python -m pip install -q -U spacy

     |████████████████████████████████| 4.0 MB 7.1 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 106 kB 44.0 MB/s 
     |████████████████████████████████| 287 kB 20.8 MB/s 
     |████████████████████████████████| 6.6 MB 22.7 MB/s 
     |████████████████████████████████| 880 kB 54.5 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
     |████████████████████████████████| 596 kB 70.2 MB/s 
     |████████████████████████████████| 462 kB 7.3 MB/s 
     |████████████████████████████████| 6.2 MB 7.3 MB/s 
     |████████████████████████████████| 653 kB 75.4 MB/s 
     |████████████████████████████████| 457 kB 61.8 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 181 kB 60.9 MB/s 
     |████████████████████████████████| 10.1 MB 30.7 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 


In [ ]:
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 460.3 MB 25 kB/s 
     |████████████████████████████████| 51 kB 176 kB/s 
     |████████████████████████████████| 3.8 MB 19.7 MB/s 
     |████████████████████████████████| 1.1 MB 32.0 MB/s 
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 10.2 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## 2. Load all libraries

In [ ]:
import numpy as np
import pandas as pd
import spacy
import nltk
import random
import re
import torch
import warnings
import torch.nn as nn

from spacy.util import minibatch, compounding
from spacy.training import Example
from pathlib import Path
from spacy.training import offsets_to_biluo_tags
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader, RandomSampler
from pprint import pprint
# from sklearn.metrics import accuracy_score, classification_report, f1_score
from seqeval.metrics import classification_report, f1_score

nltk.download('punkt')
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth=None
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 3. Load the dataset

In [ ]:
# Load data
training = pd.read_csv('training_data_with_ADR.csv')
validation = pd.read_csv('validation_data_with_ADR.csv')

In [ ]:
print(training.shape)
print(validation.shape)

(2172, 13)
(560, 13)


In [ ]:
training.head()

,Unnamed: 0,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,clean_tweets,label,ADR
0,0,331187619096588288,NaN,NaN,NaN,NaN,ofloxacin,@seefisch:oral drugs for pyelonephritis:ciprofloxacin levofloxacin tmp/smz do not use nitrofurantoin for pyelo(only cystitis)@david_medinaf,NaN,NaN,<user> : oral drugs for pyelonephritis : ciprofloxacin levofloxacin tmp / smz do not use nitrofurantoin for pyelo ( only cystitis ) <user>,0,0
1,1,332227554956161024,NaN,NaN,NaN,NaN,trazodone,happy for wellbutrin; has similar effects as adderall.. trazodone is super promising for sleep.. but abilify can cause weight gain -_-,NaN,NaN,happy for wellbutrin ; has similar effects as adderall . <repeated> trazodone is super promising for sleep . <repeated> but abilify can cause weight gain -_-,0,0
2,2,332448217490944000,NaN,NaN,NaN,NaN,lamotrigine,"@stilgarg i'm ok ty have an official diagnosis of bipolar now, feeling ok at the moment lamotrigine has been increased having monotherapy:/",NaN,NaN,"<user> i am ok ty have an official diagnosis of bipolar now , feeling ok at the moment lamotrigine has been increased having monotherapy <annoyed>",0,0
3,3,332977955754110976,NaN,NaN,NaN,NaN,cymbalta,i'm soo depressed cymbalta couldn't help me .,NaN,NaN,i am soo depressed cymbalta could not help me .,0,0
4,4,333674203331051520,NaN,NaN,NaN,NaN,seroquel,"time for my daily afternoon relaxation ritual of smoking weed, taking 2 mgs of clonazepam, and 400 mg of seroquel xr.",NaN,NaN,"time for my daily afternoon relaxation ritual of smoking weed , taking <number> mgs of clonazepam , and <number> mg of seroquel xr .",0,0


## 4. Prepare the data

In [ ]:
# Get the non-null rows
training_data = training[training.ADR == 1]
# validation_data = validation[validation.ADR == 1]
# training_data = training.copy()
validation_data = validation.copy()

# Reset index
training_data.reset_index(inplace=True, drop=True)
validation_data.reset_index(inplace=True, drop=True)

# Drop unwanted column
training_data.drop("Unnamed: 0", inplace=True, axis=1)
validation_data.drop("Unnamed: 0", inplace=True, axis=1)

# Fill in the missing values
training_data.extraction = training_data.extraction.fillna('-')
validation_data.extraction = validation_data.extraction.fillna('-')

In [ ]:
print(f"Shape of training data: {training_data.shape}")
print(f"Shape of validation data: {validation_data.shape}")

Shape of training data: (1434, 12)
Shape of validation data: (560, 12)


In [ ]:
# Ground truth labels were misleading 
# Hence create a new ground truth for start and end index

def find_start_end(dataframe):
    count = 0
    new_start = list()
    new_end = list()

    for row in dataframe.itertuples():
        if row[5] != '-': # If extraction is not empty then
            match = re.search(r"{}".format(row[5].lower().replace(")", "\)")), r"{}".format(row[7].lower()))
            if not match:
                new_start.append(int(row[2]))
                new_end.append(int(row[3]))
            else:
                if row[2] != match.start() and row[3] != match.end():
                    count += 1
                new_start.append(match.start())
                new_end.append(match.end())
        
        else: # If extraction is empty then
            new_start.append(0)
            new_end.append(0)

    print(f"Percentage of rows for which start and end index did not match is {round(count/dataframe.shape[0]*100, 4)}%")
    return new_start, new_end

In [ ]:
train_start, train_end = find_start_end(training_data)
valid_start, valid_end = find_start_end(validation_data)

Percentage of rows for which start and end index did not match is 36.4017%
Percentage of rows for which start and end index did not match is 21.0714%


In [ ]:
# Create two columns for new start and end index
training_data['new_start'] = train_start
training_data['new_end'] = train_end

validation_data['new_start'] = valid_start
validation_data['new_end'] = valid_end

In [ ]:
training_data.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,clean_tweets,label,ADR,new_start,new_end
0,342314998904786945,42.0,53.0,ADR,lost vision,cymbalta,"#cymbalta withdrawal has reached a peak, lost vision and almost crashed my car from a brain zap. thanks a zillion #elililly #bigpharma",10047522.0,vision loss,"<hashtag> cymbalta </hashtag> withdrawal has reached a peak , lost vision and almost crashed my car from a brain zap . thanks a zillion <hashtag> eli lilly </hashtag> <hashtag> big pharma </hashtag>",1,1,41,52
1,342314998904786945,11.0,21.0,ADR,withdrawal,cymbalta,"#cymbalta withdrawal has reached a peak, lost vision and almost crashed my car from a brain zap. thanks a zillion #elililly #bigpharma",10048010.0,withdrawal syndrome,"<hashtag> cymbalta </hashtag> withdrawal has reached a peak , lost vision and almost crashed my car from a brain zap . thanks a zillion <hashtag> eli lilly </hashtag> <hashtag> big pharma </hashtag>",1,1,10,20
2,342322703556038657,27.0,35.0,ADR,nauseous,cipro,i hate cipro! #antibiotic #nauseous #cf #hospitallife #cysticfibrosis,10028823.0,nauseous,i hate cipro ! <hashtag> antibiotic </hashtag> <hashtag> nauseous </hashtag> <hashtag> cf </hashtag> <hashtag> hospital life </hashtag> <hashtag> cystic fibrosis </hashtag>,1,1,27,35
3,342349802601844737,109.0,118.0,ADR,can't cum,seroquel,@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own,10021574.0,inability to orgasm,<user> reppin zoloft & seroquel since last november . i am hella gainin weight too awesome i am fat and can not cum i own,1,1,109,118
4,342349802601844737,101.0,104.0,ADR,fat,seroquel,@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own,10047896.0,weight gain,<user> reppin zoloft & seroquel since last november . i am hella gainin weight too awesome i am fat and can not cum i own,1,1,101,104


,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,clean_tweets,label,ADR,new_start,new_end
0,342314998904786945,42.0,53.0,ADR,lost vision,cymbalta,"#cymbalta withdrawal has reached a peak, lost vision and almost crashed my car from a brain zap. thanks a zillion #elililly #bigpharma",10047522.0,vision loss,"<hashtag> cymbalta </hashtag> withdrawal has reached a peak , lost vision and almost crashed my car from a brain zap . thanks a zillion <hashtag> eli lilly </hashtag> <hashtag> big pharma </hashtag>",1,1,41,52
1,342314998904786945,11.0,21.0,ADR,withdrawal,cymbalta,"#cymbalta withdrawal has reached a peak, lost vision and almost crashed my car from a brain zap. thanks a zillion #elililly #bigpharma",10048010.0,withdrawal syndrome,"<hashtag> cymbalta </hashtag> withdrawal has reached a peak , lost vision and almost crashed my car from a brain zap . thanks a zillion <hashtag> eli lilly </hashtag> <hashtag> big pharma </hashtag>",1,1,10,20
2,342322703556038657,27.0,35.0,ADR,nauseous,cipro,i hate cipro! #antibiotic #nauseous #cf #hospitallife #cysticfibrosis,10028823.0,nauseous,i hate cipro ! <hashtag> antibiotic </hashtag> <hashtag> nauseous </hashtag> <hashtag> cf </hashtag> <hashtag> hospital life </hashtag> <hashtag> cystic fibrosis </hashtag>,1,1,27,35
3,342349802601844737,109.0,118.0,ADR,can't cum,seroquel,@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own,10021574.0,inability to orgasm,<user> reppin zoloft & seroquel since last november . i am hella gainin weight too awesome i am fat and can not cum i own,1,1,109,118
4,342349802601844737,101.0,104.0,ADR,fat,seroquel,@luckystubbs reppin zoloft&amp;seroquel since last november. i'm hella gainin weight too awesome i'm fat and can't cum i own,10047896.0,weight gain,<user> reppin zoloft & seroquel since last november . i am hella gainin weight too awesome i am fat and can not cum i own,1,1,101,104


In [ ]:
# Load spacy and pipeline
nlp = spacy.load('en_core_web_trf')
# nlp = spacy.load('en_core_web_sm')
ner = nlp.get_pipe('ner')

In [ ]:
nlp.pipe_names

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
# Prepare the tweets in spacy format
TRAIN_DATA = []
VALID_DATA = []

for row in training_data.itertuples():
    TRAIN_DATA.append((row[7],{
        'entities': [(int(row[13]), int(row[14]), 'ADR')]
    }))

for row in validation_data.itertuples():
    VALID_DATA.append((row[7],{
        'entities': [(int(row[13]), int(row[14]), 'ADR')]
    }))

In [ ]:
# Tag text to BILUO format and replace the tags "L" with "I" and "U" with "B"
# for BIO labelling scheme

tags_list = list()
for text, annot in TRAIN_DATA:
    doc = nlp(text)
    tags = offsets_to_biluo_tags(doc, annot['entities'])
    bio_tags = list(map(lambda tag: tag.replace("L", "I").replace("U", "B"), tags))
    bio_tags = ['O' if tag == '-' else tag for tag in bio_tags]
    tags_list.append(bio_tags)

valid_tags_list = list()
for text, annot in VALID_DATA:
    doc = nlp(text)
    tags = offsets_to_biluo_tags(doc, annot['entities'])
    bio_tags = list(map(lambda tag: tag.replace("L", "I").replace("U", "B"), tags))
    bio_tags = ['O' if tag == '-' else tag for tag in bio_tags]
    valid_tags_list.append(bio_tags)

In [ ]:
final_train = pd.DataFrame(columns=["extraction", "tweet", "bio_tags", "label", "ADR"])
final_valid = pd.DataFrame(columns=["extraction", "tweet", "bio_tags", "label", "ADR"])

In [ ]:
final_train.extraction = training_data.extraction
final_train.bio_tags = tags_list
final_train.label = training_data.label
final_train.ADR = training_data.ADR

final_valid.extraction = validation_data.extraction
final_valid.bio_tags = valid_tags_list
final_valid.label = validation_data.label
final_valid.ADR = validation_data.ADR

In [ ]:
final_train.tweet = [[str(token) for token in nlp(tweet)] for tweet in training_data.tweet]
final_valid.tweet = [[str(token) for token in nlp(tweet)] for tweet in validation_data.tweet]

In [ ]:
final_train.head()

,extraction,tweet,bio_tags,label,ADR
0,lost vision,"[#, cymbalta, withdrawal, has, reached, a, peak, ,, lost, vision, and, almost, crashed, my, car, from, a, brain, zap, ., thanks, a, zillion, #, elililly, #, bigpharma]","[O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1
1,withdrawal,"[#, cymbalta, withdrawal, has, reached, a, peak, ,, lost, vision, and, almost, crashed, my, car, from, a, brain, zap, ., thanks, a, zillion, #, elililly, #, bigpharma]","[O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1
2,nauseous,"[i, hate, cipro, !, #, antibiotic, #, nauseous, #, cf, #, hospitallife, #, cysticfibrosis]","[O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O]",1,1
3,can't cum,"[@luckystubbs, reppin, zoloft&amp;seroquel, since, last, november, ., i, 'm, hella, gainin, weight, too, awesome, i, 'm, fat, and, ca, n't, cum, i, own]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, I-ADR, O, O]",1,1
4,fat,"[@luckystubbs, reppin, zoloft&amp;seroquel, since, last, november, ., i, 'm, hella, gainin, weight, too, awesome, i, 'm, fat, and, ca, n't, cum, i, own]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O]",1,1


In [ ]:
final_valid[final_valid.label != final_valid.ADR].shape

(72, 5)

In [ ]:
final_valid.loc[(final_valid.label == 0) & (final_valid.ADR == 1), :]

,extraction,tweet,bio_tags,label,ADR
7,-,"[@deadfanclub, @danadelany, i, have, a, penicillin/, keflex, allergy, so, my, dr, writes, me, 10, days, of, zpack, or, i, 'd, still, be, sick, too, !, sometimes, cipro]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
12,-,"[@lanternlizard, cymbalta, switch, is, part, of, what, made, me, flip, ., but, any, snri, with, family, hx, of, mania, puts, you, at, risk, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
20,-,"[@autumnuh_beee, i, was, on, celexa, ,, which, made, me, sleepy, ., i, switched, to, effexor, ., it, has, norepinephrine, in, it, ,, so, it, jump, starts, me, in, the, mornin]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
21,-,"[i, feel, so, pill, -, ill, again, -, decided, to, seriously, cut, down, to, just, weekly, enbrel, and, metho, alongside, daily, nsaids, #, rheum, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
22,-,"[rt, @dratp, :, @lisabinkc, @trainingwithra, @dsymons, enbrel, also, available, as, prefilled, syringe, ,, much, gentler, ., pen, leaves, bruises, !, #, rheum]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
26,-,"[i, have, some, seriously, disturbing, dreams, ., thanks, #, fluoxetine, thanks, a, freaking, lot, ., well, i, guess, it, 's, better, than, being, #, suicidal, .., only, just]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
30,-,"[will, still, be, on, the, geodon, ,, but, will, also, be, taking, a, low, -, dose, alprazolam, as, needed, for, the, sudden, anxiety, attacks, i, ve, been, getting]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
33,-,"[@showbizmom48, mri, showed, lesions, in, my, noggin, :, s, got, to, have, eeg, to, see, if, its, ms, or, humira, doing, it.so, more, tests, n, messing, about, lolxx]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
57,-,"[accidentally, asphyxiating, yourself, trying, to, suck, a, throat, lozenge, when, you, have, a, stuffed, nose, ., #, that]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1
81,-,"[@iamcarrieeeeee, that, 's, what, i, get, #, bipolar2, but, the, bloody, quetiapine, made, me, so, bloody, miserable, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1


In [ ]:
final_valid.loc[(final_valid.label == 1) & (final_valid.ADR == 0), :]

,extraction,tweet,bio_tags,label,ADR
0,allergies,"[do, you, have, any, medication, allergies, ?, "", asthma, !, !, !, "", me, :, "", ........, "", pt, :, "", no, wait, ., avelox, ,, that, 's, it, !, "", "", so, no, other, allergies, ?, "", "", right, !, "", *, cont]","[O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,0
1,HURT YOUR Liver,"[@ashleylvivian, if, #, avelox, has, hurt, your, liver, ,, avoid, tylenol, always, ,, as, it, further, damages, liver, ,, eat, grapefruit, unless, taking, cardiac, drugs]","[O, O, O, O, O, B-ADR, I-ADR, I-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,0
23,space cadet,"[note, to, self, :, do, not, take, #, fluoxetine, and, #, aleved, together, ., i, 'm, a, space, cadet, today, !, , #, druginteractions]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O]",1,0
47,rashes,"[so, while, i, 'm, letting, the, thoughts, of, lamictal, /, fatal, rashes, /, tonsillectomy, /, spinal, tap, settle, ,, i, 'm, going, to, go, rescue, a, budgie, .]","[O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,0
83,allergic,"[(, triggering, ), so, ..., i, 'm, taking, quetiapine, and, tramadol, pre, -, cutting, ., i, 'm, allergic, to, tramadol, ..., this, should, be, interesting, #, selfharm]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O]",1,0
114,dependence,"[but, i, do, n't, want, to, take, an, ambien, or, trazodone, or, anything, because, i, dint, wanna, restart, my, dependence, upon, them]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O]",1,0
172,parody of an actual human being,"[wellbutrin, ,, zyprexa, ,, xanax, ,, effexor, ,, lamotrigine, ,, valium, ., every, day, ., i, 'm, not, even, a, person, anymore, ., i, 'm, a, parody, of, an, actual, human, being, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, I-ADR, I-ADR, I-ADR, I-ADR, O]",1,0
184,tired daytimes,"[tonight, i, am, taking, my, #, venlafaxine, in, the, evening, in, pursuit, of, better, sleep, and, less, tired, daytimes, ., wish, me, luck, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O]",1,0
303,headache,"[nicotine, is, literally, poison, ., but, sometimes, you, got, ta, find, out, the, hard, way, ., #, headache]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR]",1,0
304,poison,"[nicotine, is, literally, poison, ., but, sometimes, you, got, ta, find, out, the, hard, way, ., #, headache]","[O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,0


In [ ]:
final_valid_zero = final_valid[final_valid.ADR == 0]
final_valid = final_valid[final_valid.ADR == 1]

In [ ]:
final_valid_zero.reset_index(inplace=True, drop=True)
final_valid.reset_index(inplace=True, drop=True)

In [ ]:
print(final_valid_zero.shape)
print(final_valid.shape)

(169, 5)
(391, 5)


In [ ]:
# Dictionary to keep track of labels and index mapping 
all_tags = ','.join(final_train.bio_tags.apply(lambda x: ','.join(x)))
unique_tags = np.unique(all_tags.split(','))
labels_to_ids = {k: v for v, k in enumerate(unique_tags)}
ids_to_labels = {v: k for v, k in enumerate(unique_tags)}

In [ ]:
pprint(ids_to_labels)
pprint(labels_to_ids)

{0: 'B-ADR', 1: 'I-ADR', 2: 'O'}
{'B-ADR': 0, 'I-ADR': 1, 'O': 2}


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1
EPOCHS = 5
LEARNING_RATE = 5e-05
MAX_GRAD_NORM = 10
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

Downloading:   0%|          | 0.00/313 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

In [ ]:
# encoding = tokenizer(final_train.tweet.iloc[0],
#                         is_split_into_words=True,
#                         return_offsets_mapping=True, 
#                         padding='max_length', 
#                         truncation=True, 
#                         max_length=128)

In [ ]:
# for key, value in encoding.items():
#     print(key, value)

In [ ]:
# for token, offset in zip(tokenizer.convert_ids_to_tokens(encoding["input_ids"]), encoding['offset_mapping']):
#   print('{0:10} {1}'.format(token, offset))

In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        
        tweet = self.data.tweet[index]
        # text = nlp(self.data.tweet[index].strip())
        # tweet = [token for token in text]
        bio_tags = self.data.bio_tags[index]
        encoding = self.tokenizer(tweet,
                                 is_split_into_words=True,
                                 return_offsets_mapping=True, 
                                 padding='max_length', 
                                 truncation=True, 
                                 max_length=self.max_len)
        
        labels = [labels_to_ids[label] for label in bio_tags]
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
            if mapping[0] == 0 and mapping[1] == 0:
                # print(0,0)
                continue
            if mapping[0] == 0 and mapping[1] != 0:
                encoded_labels[idx] = labels[i]
                # print('first match')
                i += 1
            else:
                # print('next match')
                # encoded_labels[idx] = encoded_labels[idx-1]
                continue
        
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item, index
    
    def __len__(self):
        return self.len

In [ ]:
training_set = dataset(final_train, tokenizer, MAX_LEN)
testing_set = dataset(final_valid, tokenizer, MAX_LEN)

In [ ]:
training_set[0]

({'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0]),
  'input_ids': tensor([  101,   108,   172, 17162, 26443,  1161, 10602,  1144,  1680,   170,
           4709,   117,  1575,  4152,  1105,  1593,  7573,  1139,  1610,  1121,
            170,  3575,   195, 11478,   119,  5438,   170,   195,  7956,  1988,
            108,  8468, 18575,  6071,   108,  1992, 20695, 10841,   102,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(testing_set[0][0]["input_ids"]), testing_set[0][0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
apparently  2
,           2
b           2
##ac        -100
##lo        -100
##fen       -100
greatly     2
ex          2
##ace       -100
##rb        -100
##ates      -100
the         2
"           2
ad          0
"           2
part        2
of          2
my          2
ad          2
##h         -100
##d         -100
.           2
average     2
length      2
of          2
focus       2
today       2
:           2
about       2
30          2
seconds     2
.           2
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from transformers import AutoModelForTokenClassification

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels=len(labels_to_ids)).to(device)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [ ]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [ ]:
print(f"The model has {count_parameter(model):,} trainable parameters.")

The model has 107,721,987 trainable parameters.


In [ ]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
outputs.loss

tensor(1.0514, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch, clip=1.0):
    tr_loss, tr_f1 = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        # print(idx)
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        optimizer.zero_grad()
        md = model(input_ids=ids, attention_mask=mask, labels=labels)
        
        loss, tr_logits = md.loss, md.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
        # torch.argmax(outputs.logits, axis=2)

        mask_label = labels.view(-1)
        mask_labels = torch.masked_select(mask_label, active_accuracy)
        
        mask_prediction = torch.argmax(tr_logits, axis=2).view(-1)
        mask_predictions = torch.masked_select(mask_prediction, active_accuracy)
        # mask_predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        # print(mask_labels)
        # print('-'*5)
        # print(mask_predictions)
        
        labels, predictions = list(), list()
        for label in mask_labels.tolist():
          labels.append(ids_to_labels.get(label, 'O'))
        
        for pred in mask_predictions.tolist():
          predictions.append(ids_to_labels.get(pred, 'O'))

        tr_labels.append(labels)
        tr_preds.append(predictions)

        tmp_tr_f1 = f1_score([labels], [predictions], average='micro')
        tr_f1 += tmp_tr_f1
    
        loss.backward()

        # gradient clipping
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # backward pass
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_f1 = tr_f1 / nb_tr_steps
    print(f"Training loss epoch {epoch+1}: {epoch_loss}")
    print(f"Training F1 score epoch {epoch+1}: {tr_f1}\n")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.8658053278923035
Training loss per 100 training steps: 0.3821342797894584
Training loss per 100 training steps: 0.3817507125418726
Training loss per 100 training steps: 0.3597431673306316
Training loss per 100 training steps: 0.35602486327504207
Training loss per 100 training steps: 0.3425280800583715
Training loss per 100 training steps: 0.344474541582787
Training loss per 100 training steps: 0.3373578395030213
Training loss per 100 training steps: 0.3290825623300359
Training loss per 100 training steps: 0.32361521310876895
Training loss per 100 training steps: 0.3159818239782199
Training loss per 100 training steps: 0.31371355769613574
Training loss per 100 training steps: 0.31016734286427006
Training loss per 100 training steps: 0.304167929898087
Training loss per 100 training steps: 0.3014121195396141
Training loss epoch 1: 0.29989951051855424
Training F1 score epoch 1: 0.2427608421332272

Training epoch: 2
Training loss per

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch, index in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            md = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss, eval_logits = md.loss, md.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            # mask_labels = labels.view(-1)
            # mask_predictions = torch.argmax(eval_logits, axis=2).view(-1)

            mask_label = labels.view(-1)
            mask_labels = torch.masked_select(mask_label, active_accuracy)
            
            mask_prediction = torch.argmax(eval_logits, axis=2).view(-1)
            mask_predictions = torch.masked_select(mask_prediction, active_accuracy)

            # mask_labels = torch.masked_select(flattened_targets, active_accuracy)
            # mask_predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            labels, predictions = list(), list()
            for label in mask_labels.tolist():
            #   labels.append(ids_to_labels[label])
              labels.append(ids_to_labels.get(label, 'O'))
            
            for pred in mask_predictions.tolist():
            #   predictions.append(ids_to_labels[pred])
              predictions.append(ids_to_labels.get(pred, 'O'))

            eval_labels.append(labels)
            eval_preds.append(predictions)
            
            tmp_eval_accuracy = f1_score([labels], [predictions], average='micro')
            eval_accuracy += tmp_eval_accuracy

    # labels = [[ids_to_labels[id.item()] for id in lab] for lab in eval_labels]
    # predictions = [[ids_to_labels[id.item()] for id in lab] for lab in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"\nValidation Loss: {eval_loss}")
    print(f"Validation F1 score: {eval_accuracy}")

    return eval_labels, eval_preds

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.09865045547485352
Validation loss per 100 evaluation steps: 0.4181884930487538
Validation loss per 100 evaluation steps: 0.3918832171434972
Validation loss per 100 evaluation steps: 0.3901637492307301

Validation Loss: 0.3727611526246945
Validation F1 score: 0.20528559249786862


In [ ]:
# final_valid_zero.head(2)

In [ ]:
label_list = final_valid_zero.bio_tags.tolist()
pred_list = list()
for value in final_valid_zero.itertuples():
    temp = ['O'] * len(value[2])
    pred_list.append(temp)

In [ ]:
print(len(label_list))
print(len(pred_list))

169
169


In [ ]:
assert len(label_list) == len(pred_list)

In [ ]:
labels.extend(label_list)
predictions.extend(pred_list)

In [ ]:
assert len(labels) == len(predictions)

In [ ]:
print(classification_report(eval_labels, eval_preds))

              precision    recall  f1-score   support

         ADR       0.27      0.32      0.29       357

   micro avg       0.27      0.32      0.29       357
   macro avg       0.27      0.32      0.29       357
weighted avg       0.27      0.32      0.29       357



In [ ]:
from seqeval.scheme import IOB2

In [ ]:
print(classification_report(eval_labels, eval_preds, mode='strict', scheme=IOB2))

              precision    recall  f1-score   support

         ADR       0.28      0.30      0.29       357

   micro avg       0.28      0.30      0.29       357
   macro avg       0.28      0.30      0.29       357
weighted avg       0.28      0.30      0.29       357



In [ ]:
torch.save(model.state_dict(), "biobert_ner.pt")

### Load the saved model

In [ ]:
# Loading the saved model
output_model = 'biobert_ner_new.pt'

model_test = AutoModelForTokenClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels=len(labels_to_ids)).to(device)
model_test.load_state_dict(torch.load(output_model, map_location=device))

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

<All keys matched successfully>

In [ ]:
model_test.eval()
eval_preds, eval_labels = [], []
save_ids = list()
indexes = list()

with torch.no_grad():
    for batch, index in testing_loader:
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)
        
        md = model_test(input_ids=ids, attention_mask=mask, labels=labels)
        loss, eval_logits = md.loss, md.logits

        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = eval_logits.view(-1, model_test.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

        mask_label = labels.view(-1)
        mask_labels = torch.masked_select(mask_label, active_accuracy)
        
        mask_prediction = torch.argmax(eval_logits, axis=2).view(-1)
        mask_predictions = torch.masked_select(mask_prediction, active_accuracy)
        
        labels, predictions = list(), list()
        for label in mask_labels.tolist():
          labels.append(ids_to_labels.get(label, 'O'))
        
        for pred in mask_predictions.tolist():
          predictions.append(ids_to_labels.get(pred, 'O'))

        eval_labels.append(labels) 
        eval_preds.append(predictions)
        save_ids.append(torch.masked_select(ids.view(-1), active_accuracy))
        indexes.append(index)

In [ ]:
label_list = final_valid_zero.bio_tags.tolist()
pred_list = list()
for value in final_valid_zero.itertuples():
    temp = ['O'] * len(value[2])
    pred_list.append(temp)

In [ ]:
print(len(label_list))
print(len(pred_list))

169
169


In [ ]:
assert len(label_list) == len(pred_list)

In [ ]:
eval_labels.extend(label_list)
eval_preds.extend(pred_list)

In [ ]:
assert len(eval_labels) == len(eval_preds)

In [ ]:
print(classification_report(eval_labels, eval_preds))

              precision    recall  f1-score   support

         ADR       0.27      0.32      0.29       357

   micro avg       0.27      0.32      0.29       357
   macro avg       0.27      0.32      0.29       357
weighted avg       0.27      0.32      0.29       357



In [ ]:
from seqeval.scheme import IOB2

In [ ]:
print(classification_report(eval_labels, eval_preds, mode='strict', scheme=IOB2))

              precision    recall  f1-score   support

         ADR       0.28      0.30      0.29       357

   micro avg       0.28      0.30      0.29       357
   macro avg       0.28      0.30      0.29       357
weighted avg       0.28      0.30      0.29       357



In [ ]:
text = list()
tag = list()

for id, label in zip(save_ids, eval_labels):
    bool_list = id != 0
    final_id = torch.masked_select(id, bool_list)
#   print(final_id)
#   print(label)
#   print(tokenizer.convert_ids_to_tokens(final_id))

    text.append(tokenizer.convert_ids_to_tokens(final_id))
    tag.append(label)
  # for token, labl in zip(tokenizer.convert_ids_to_tokens(final_id), label):
  #   print('{0:10}  {1}'.format(token, labl))
  #   break

In [ ]:
# for i, j in zip(text, tag):
#     print()

In [ ]:
# for i, j in zip(text, tag):
    # print(len(i), len(j))
    # print(i, j)
    # assert len(i) == len(j)

In [ ]:
len(final_valid.iloc[375].tweet)

27

In [ ]:
extracted_text = list()
for twe, ta in zip(text, tag):
    temp = []
    for tw, tg in zip(twe, ta):
        if 'B' in tg:
            temp.append(tw)
        elif 'I' in tg:
            temp[-1] = ' '.join([temp[-1], tw])
    extracted_text.append(temp)

In [ ]:
len(extracted_text)

391

In [ ]:
new_indexes = [index.tolist()[0] for index in indexes]

In [ ]:
final_valid.loc[new_indexes, 'pred_extraction'] = extracted_text
final_valid.loc[new_indexes, 'pre_bio'] = tag

In [ ]:
pred_list = list()
pred_ext = list()
for value in final_valid_zero.itertuples():
    temp = ['O'] * len(value[2])
    pred_list.append(temp)
    pred_ext.append([])

In [ ]:
final_valid_zero['pred_extraction'] = pred_ext
final_valid_zero['pre_bio'] = pred_list

In [ ]:
final_final_valid = pd.concat([final_valid, final_valid_zero])

In [ ]:
final_final_valid.to_excel("validation_with_output.xlsx")
final_final_valid.to_csv("validation_with_output.csv")

## The end

In [ ]:
a

In [ ]:
final_valid_zero.shape

(169, 5)

In [ ]:
pd.concat([final_valid, final_valid_zero])

In [ ]:
final_valid

,extraction,tweet,bio_tags,label,ADR,pred_extraction,pre_bio
0,AD,"[apparently, ,, baclofen, greatly, exacerbates, the, "", ad, "", part, of, my, adhd, ., average, length, of, focus, today, :, about, 30, seconds, .]","[O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1,[ad],"[O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,focus,"[apparently, ,, baclofen, greatly, exacerbates, the, "", ad, "", part, of, my, adhd, ., average, length, of, focus, today, :, about, 30, seconds, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O]",1,1,[focus],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O]"
2,died,"[pt, of, mine, died, from, cipro, rt, @ciproispoison, :, @gastromom, if, only, more, doctors, thought, like, you, !, i, lost, my, entire, life, to, 12, cipro, pills]","[O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1,[died],"[O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,died,"[@gastromom, the, only, pt, of, mine, who, ever, died, was, one, age, 21, profound, autism, chronic, underwt, &, amp, ;, gi, issues, ,, given, lots, of, cipro, ., terrible]","[O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1,[died],"[O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,tendon damage,"[owww, ., i, hurt, my, foot, ., &, amp, ;, am, concerned, ., 1st, warning, on, cipro, is, tendon, damage, ., :-/, it, certainly, was, n't, a, rupture, ., i, 'm, sure, i, 'm, just, paranoid, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1,[tend damage],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
5,-,"[@deadfanclub, @danadelany, i, have, a, penicillin/, keflex, allergy, so, my, dr, writes, me, 10, days, of, zpack, or, i, 'd, still, be, sick, too, !, sometimes, cipro]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1,[],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,dreams,"[@bilgeebiri, oh, robitussin, dreams, are, notorious, ., although, what, will, really, turn, 'em, into, a, bunuel, movie, is, cipro, .]","[O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1,[dreams],"[O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,tendon rupture,"[#, eds, friends, !, anybody, taken, #, cipro, ?, (, antibiotic, ), complications, ?, ?, big, side, effect, is, tendon, rupture, ..., figured, my, dr, would, know, better, ?]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O, O, O, O]",1,1,[tend r],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O, O, O, O]"
8,-,"[@lanternlizard, cymbalta, switch, is, part, of, what, made, me, flip, ., but, any, snri, with, family, hx, of, mania, puts, you, at, risk, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,1,[],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,withdrawal,"[@enlightenmentor, umm, ,, are, you, being, a, smart, -, ass, ?, do, nt, mess, with, me, right, now, ., going, through, cymbalta, withdrawal, i, think, &, amp, ;, i, m, very, depressed, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O]",1,1,[withdrawal],"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O]"


In [ ]:
final_valid_zero

,extraction,tweet,bio_tags,label,ADR
0,allergies,"[do, you, have, any, medication, allergies, ?, "", asthma, !, !, !, "", me, :, "", ........, "", pt, :, "", no, wait, ., avelox, ,, that, 's, it, !, "", "", so, no, other, allergies, ?, "", "", right, !, "", *, cont]","[O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,0
1,HURT YOUR Liver,"[@ashleylvivian, if, #, avelox, has, hurt, your, liver, ,, avoid, tylenol, always, ,, as, it, further, damages, liver, ,, eat, grapefruit, unless, taking, cardiac, drugs]","[O, O, O, O, O, B-ADR, I-ADR, I-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,0
2,-,"[@joshuagates, you, forgot, the, immodium, for, when, the, cipro, kills, all, the, good, bacteria, ., :), #, travelskills]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,0
3,-,"[@illtreaturite1, fluox, a, friend, 4, years, ,, but, switch, to, cymbalta, 4, anti, -, depress, /, pain, control, ., i, ocd, worry, ., ca, n't, stop, running, mind]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,0
4,space cadet,"[note, to, self, :, do, not, take, #, fluoxetine, and, #, aleved, together, ., i, 'm, a, space, cadet, today, !, , #, druginteractions]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O]",1,0
5,rashes,"[so, while, i, 'm, letting, the, thoughts, of, lamictal, /, fatal, rashes, /, tonsillectomy, /, spinal, tap, settle, ,, i, 'm, going, to, go, rescue, a, budgie, .]","[O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,0
6,allergic,"[(, triggering, ), so, ..., i, 'm, taking, quetiapine, and, tramadol, pre, -, cutting, ., i, 'm, allergic, to, tramadol, ..., this, should, be, interesting, #, selfharm]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O]",1,0
7,dependence,"[but, i, do, n't, want, to, take, an, ambien, or, trazodone, or, anything, because, i, dint, wanna, restart, my, dependence, upon, them]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O]",1,0
8,-,"[do, u, know, what, medications, are, r, for, bipolar, depression, ?, currently, #, fda, approved, lamotrigine, aka, #, lamictal]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",0,0
9,parody of an actual human being,"[wellbutrin, ,, zyprexa, ,, xanax, ,, effexor, ,, lamotrigine, ,, valium, ., every, day, ., i, 'm, not, even, a, person, anymore, ., i, 'm, a, parody, of, an, actual, human, being, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, I-ADR, I-ADR, I-ADR, I-ADR, O]",1,0


In [ ]:
validation_data.label.value_counts()

1    365
0    195
Name: label, dtype: int64

In [ ]:
final_valid.head()

,extraction,tweet,bio_tags,label,ADR
0,AD,"[apparently, ,, baclofen, greatly, exacerbates, the, "", ad, "", part, of, my, adhd, ., average, length, of, focus, today, :, about, 30, seconds, .]","[O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1
1,focus,"[apparently, ,, baclofen, greatly, exacerbates, the, "", ad, "", part, of, my, adhd, ., average, length, of, focus, today, :, about, 30, seconds, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O]",1,1
2,died,"[pt, of, mine, died, from, cipro, rt, @ciproispoison, :, @gastromom, if, only, more, doctors, thought, like, you, !, i, lost, my, entire, life, to, 12, cipro, pills]","[O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1
3,died,"[@gastromom, the, only, pt, of, mine, who, ever, died, was, one, age, 21, profound, autism, chronic, underwt, &, amp, ;, gi, issues, ,, given, lots, of, cipro, ., terrible]","[O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1
4,tendon damage,"[owww, ., i, hurt, my, foot, ., &, amp, ;, am, concerned, ., 1st, warning, on, cipro, is, tendon, damage, ., :-/, it, certainly, was, n't, a, rupture, ., i, 'm, sure, i, 'm, just, paranoid, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",1,1


In [ ]:
validation_data.shape

(560, 14)

In [ ]:
validation_data[validation_data.label != validation_data.ADR].label.value_counts()

0    49
1    23
Name: label, dtype: int64

In [ ]:
validation_data.iloc[10]

tweet_id                                                                                                                                                                                              352217412046823425
begin                                                                                                                                                                                                               84.0
end                                                                                                                                                                                                                 98.0
type                                                                                                                                                                                                                 ADR
extraction                                                                                                                          

In [ ]:
index = 0
for l, p in zip(labels, predictions):
    if (('B-ADR' not in l) and ('B-ADR' in p)) or (('B-ADR' in l) and ('B-ADR' not in p)):
        print(index, l)
        print(p)
    index += 1

0 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [ ]:
validation_

In [ ]:
validation_data.head()

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term,clean_tweets,label,ADR,new_start,new_end
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!!!"" me: ""........"" pt: ""no wait. avelox, that's it!"" ""so no other allergies?"" ""right!"" *cont",10013661.0,drug allergy,"do you have any medication allergies ? "" asthma ! <repeated> "" me : "" . <repeated> "" pt : "" no wait . avelox , that ' s it ! "" "" so no other allergies ? "" "" right ! "" * cont",1,0,27,36
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver, avoid tylenol always, as it further damages liver, eat grapefruit unless taking cardiac drugs",10024668.0,liver damage,"<user> if <hashtag> a velox </hashtag> has hurt your liver , avoid tylenol always , as it further damages liver , eat grapefruit unless taking cardiac drugs",1,0,30,45
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003731.0,attention deficit disorder,"apparently , baclofen greatly exacerbates the "" ad "" part of my adhd . average length of focus today : about <number> seconds .",1,1,46,48
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""ad"" part of my adhd. average length of focus today: about 30 seconds.",10003738.0,attention impaired,"apparently , baclofen greatly exacerbates the "" ad "" part of my adhd . average length of focus today : about <number> seconds .",1,1,85,90
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: @gastromom if only more doctors thought like you! i lost my entire life to 12 cipro pills,10011906.0,death,pt of mine died from cipro rt <user> : <user> if only more doctors thought like you ! i lost my entire life to <number> cipro pills,1,1,11,15
